In [2]:
import torch

from pkg.datasets import IDSDataset
from pkg.lit_sad_ae import LitIDSAE_SAD
from pkg.lit_ae import LitIDSAE

In [3]:
from torchvision import transforms, utils
from tqdm import tqdm
import pickle
import pandas as pd
# Маппинги айпи адресов
with open('Data/sip_map.pkl', 'rb') as f:
    sip_map = pickle.load(f)
with open('Data/dip_map.pkl', 'rb') as f:
    dip_map = pickle.load(f)

In [4]:
autoencoder = LitIDSAE_SAD(sip_map, dip_map, 128, 512, use_category=False)

In [5]:
PATH = 'Models/autoencoder-final-epoch=26-step=68302.ckpt'
base_enc = LitIDSAE.load_from_checkpoint(PATH, sip_map=sip_map, dip_map=dip_map, use_category=False, map_location=torch.device('cpu'))

In [6]:
autoencoder.init_encoder(base_enc.state_dict())

In [7]:
# Use min max stats from whole dataset
ds_train = IDSDataset('Data/all_train.csv', sip_map, dip_map, only_benign=True, transform=True,
                     stat=pd.read_csv('Data/all_stat.csv').rename({0:'count', 1:'mean', 2:'std', 3:'min', 4:'25%', 5:'50%', 6:'75%', 7:'max'}))
#ds_test = IDSDataset('Data/all_test.csv', sip_map, dip_map, only_benign=True, transform=True)

In [8]:
from torch.utils.data import DataLoader
BATCH_SIZE = 3 ** 5
train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=4)

In [9]:
autoencoder = autoencoder.to(torch.device('cpu'))
autoencoder.init_center_c(train_loader)

In [10]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger(
    save_dir='exps',
    name='ae-sad'
)
trainer = pl.Trainer(gpus=[0],logger=logger, max_epochs=5)

MisconfigurationException: You requested GPUs: [0]
 But your machine only has: []

In [ ]:
BATCH_SIZE = 3 ** 5
ds_train = IDSDataset('Data/all_train.csv', sip_map, dip_map, only_benign=False, transform=True,
                                     stat=pd.read_csv('Data/all_stat.csv').rename({0:'count', 1:'mean', 2:'std', 3:'min', 4:'25%', 5:'50%', 6:'75%', 7:'max'}))
train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=4)

In [ ]:
trainer.fit(autoencoder, train_loader)

In [ ]:
batch = next(iter(train_loader))

In [ ]:
categorical = batch[0]
continuous =  batch[1].unsqueeze(-2)
labels = batch[2].unsqueeze(-1)
x = autoencoder.categorical_emb(categorical, continuous)
mu_p, logvar_p = autoencoder(x)

dist = 0.5 * torch.sum(autoencoder.v_c - logvar_p  - 1 + (mu_p - autoencoder.m_c) * \
                    (1 / autoencoder.v_c.exp()) * (mu_p - autoencoder.m_c) + \
                    (1 / autoencoder.v_c.exp()) * logvar_p.exp(), dim = -1)
# losses = torch.where(labels == 0, dist, autoencoder.eta * ((dist + autoencoder.eps) ** -1))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56316f98-13a2-452c-be31-8997c1e2f7a7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>